<a href="https://colab.research.google.com/github/mokshanandam/CodSoft/blob/main/TOMATO_PLANT_DISEACE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import zipfile
import os

# Path to the uploaded zip file
zip_file_path = '/archive (7).zip'

# Directory where the files will be extracted
extract_dir = '/content/tomato_leaf_disease_dataset'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# List extracted files and directories
extracted_files = os.listdir(extract_dir)
print(extracted_files)


['tomato']


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Include validation split

# Create training and validation data generators
train_generator = train_datagen.flow_from_directory(
    extract_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    extract_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 8788 images belonging to 1 classes.
Found 2196 images belonging to 1 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=25,
    callbacks=[early_stopping]
)


Epoch 1/25
274/274 ━━━━━━━━━━━━━━━━━━━━ 521s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/25
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 948us/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/25
274/274 ━━━━━━━━━━━━━━━━━━━━ 557s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/25
274/274 ━━━━━━━━━━━━━━━━━━━━ 2s 898us/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/25
140/274 ━━━━━━━━━━━━━━━━━━━━ 3:54 2s/step - accuracy: 1.0000 - loss: 0.0000e+00

In [8]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_acc * 100:.2f}%')


69/69 ━━━━━━━━━━━━━━━━━━━━ 35s 497ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Validation Accuracy: 100.00%


In [9]:
model.save('tomato_leaf_disease_detector.h5')


In [16]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess a single image for prediction
img_path = '/content/1b1762b5-7aef-463c-ab99-bcc49246407a___UF.GRC_YLCV_Lab 01470.JPG'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

# Mapping the predicted class back to the disease label
class_labels = list(train_generator.class_indices.keys())
print(f'Predicted Disease: {class_labels[predicted_class]}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Disease: tomato


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section